In [1]:
# Installazione pacchetti (da eseguire solo la prima volta)
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q qwen-omni-utils -U
!pip install -q bitsandbytes -U
!pip install -q accelerate
!pip install -q pandas # pandas for CSV operations, though not strictly needed here for basic csv

# Import
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import re
import gc
import csv
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B"
# --- Preparazione della lista dei prompt dal CSV ---
# Percorso del file CSV
csv_file = "/kaggle/input/analisi-audio-per-canzone/analisi_audio_per_canzone (2).csv"

# Lista per contenere tutti i prompt generati
prompt_list = []
# Lista per salvare i dati per il CSV dei risultati finali
results_for_csv = []

try:
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)  # Skip header

        for i, row in enumerate(reader, start=1):
            song_name = row[0]
            descrizioni = row[1:7] # The 5 fragments and the total description

            # Construct the prompt
            prompt = f"Nome Canzone: {song_name}\n\n"
            prompt += "Dati questi frammenti descrittivi di una canzone, valuta su una scala da 0 a 255 quanto ciascuno dei seguenti stati emotivi è evocato dalla descrizione:\n"
            prompt += "agitazione/movimento (R), calma (G), tristezza (B).\n\n"

            # *** NUOVE ISTRUZIONI PER IL FORMATO E IL COMPORTAMENTO ***
            prompt += "Restituisci **ESATTAMENTE 6 triple RGB COMPLETE**, una dopo l'altra, su una SINGOLA riga. \n"
            prompt += "Non includere nessuna spiegazione, numerazione, elenchi puntati o altro testo aggiuntivo. Non usare la modalità di pensiero ('thinking mode').\n"
            prompt += "Inizia la riga con 'RGB:' e poi le 6 triple. Ogni tripla deve contenere esattamente tre valori separati da virgole.\n\n"
            prompt += "Ecco l'ESEMPIO ESATTO del formato desiderato (solo 6 triple complete, su una riga):\n"
            prompt += "RGB: (R1, G1, B1) (R2, G2, B2) (R3, G3, B3) (R4, G4, B4) (R5, G5, B5) (R6, G6, B6)\n\n"
            # *** FINE NUOVE ISTRUZIONI ***
            prompt += "Ecco i frammenti:\n"
            prompt += "RGB: (valore_movimento, valore_calma, valore_tristezza)\n\n" # Reinforce exact format
            prompt += "Ecco i frammenti:\n"

            for idx_desc, desc in enumerate(descrizioni):
                label = f"Frammento {idx_desc+1}"
                if idx_desc == 5:
                    label = "Descrizione Totale"
                prompt += f"- {label}: {desc.strip()}\n"

            prompt_list.append({'song_name': song_name, 'prompt_text': prompt})

except FileNotFoundError:
    print(f"Errore: Il file CSV '{csv_file}' non è stato trovato.")
    print("Assicurati che il file si trovi nel percorso specificato.")
    exit()

# --- Fine preparazione della lista dei prompt ---


# Parametri modello

compute_dtype = torch.float16

# Configurazione quantizzazione 4-bit
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype
)

# Caricamento tokenizer e modello
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=quant_config,
    device_map="auto",
    low_cpu_mem_usage=True,
    torch_dtype=compute_dtype
)

# Loop sui prompt
for idx, entry in enumerate(prompt_list, start=1):
    song_name = entry['song_name']
    prompt_text = entry['prompt_text']

    print(f"\n=== Elaborazione canzone {idx}: {song_name} ===\n")

    chat = [{"role": "user", "content": prompt_text}]
    chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, enable_thinking=False)

    inputs = tokenizer(chat_text, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        temperature=0.1,
        num_return_sequences=1,
    )

    response_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    print("Risposta generata dal modello:\n")
    print(response_text)

    # --- MODIFICA REGEX ---
    # Questa espressione regolare cerca qualsiasi sequenza di (numero, numero, numero)
    # catturando i tre numeri.
    rgb_value_pattern = re.compile(r"\((\d{1,3}),\s*(\d{1,3}),\s*(\d{1,3})\)")
    found_rgb_matches = rgb_value_pattern.findall(response_text)
    # --- FINE MODIFICA ---

    current_song_rgb_values = [song_name] # Inizia con il nome della canzone

    # Popola fino a 6 valori RGB. Se ne trova meno, i rimanenti saranno (0,0,0).
    for i in range(6): # Ci aspettiamo esattamente 6 RGB
        if i < len(found_rgb_matches):
            try:
                # Accedi ai gruppi catturati direttamente dalla tupla
                r, g, b = int(found_rgb_matches[i][0]), int(found_rgb_matches[i][1]), int(found_rgb_matches[i][2])
                r, g, b = max(0, min(255, r)), max(0, min(255, g)), max(0, min(255, b))
                current_song_rgb_values.append(f"rgb({r}, {g}, {b})")
            except ValueError:
                # Questo gestirà anche casi come (200) dove manca il resto dei numeri
                print(f"Errore nel parsing RGB per il match {found_rgb_matches[i] if i < len(found_rgb_matches) else 'inatteso'}. Usando nero.")
                current_song_rgb_values.append("rgb(0, 0, 0)")
        else:
            # Se non trova abbastanza RGB, aggiunge il nero
            current_song_rgb_values.append("rgb(0, 0, 0)")
            print(f"Avviso: Meno di 6 triple RGB complete trovate per {song_name}. Aggiungendo nero per il frammento {i+1}.")

    results_for_csv.append(current_song_rgb_values)

    # Pulisci la memoria GPU e CPU dopo ogni elaborazione
    del inputs, outputs # <--- 'outputs' è definito qui, all'interno del loop
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

# --- Salvataggio finale dei dati ---

# Saving RGBs only to a separate CSV file
print("\n--- Salvataggio dei colori RGB in CSV ---")
output_csv_file = "valutazione_rgb_segmenti.csv"
csv_header = ['Nome Canzone', 'Frammento 1', 'Frammento 2', 'Frammento 3', 'Frammento 4', 'Frammento 5', 'Descrizione Totale']

with open(output_csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(csv_header)
    writer.writerows(results_for_csv)
print(f"Colori RGB salvati in '{output_csv_file}'")

print("\nProcessing complete. No visualizations or JSON output were generated as per your request.")
print("The RGB values are saved in the CSV file.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


2025-06-11 14:53:11.030928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749653591.055634    1119 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749653591.060667    1119 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Elaborazione canzone 1: 000207.mp3 ===

Risposta generata dal modello:

RGB: (170, 30, 50) (190, 80, 40) (100, 150, 80) (120, 130, 60) (80, 140, 100) (150, 110, 60)


...


=== Elaborazione canzone 60: 000540.mp3 ===

Risposta generata dal modello:

RGB: (120, 180, 60) (150, 190, 50) (130, 170, 65) (140, 160, 70) (180, 140, 80) (200, 120, 90)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Elaborazione canzone 61: 000897.mp3 ===

Risposta generata dal modello:

RGB: (220, 50, 30) (150, 70, 40) (200, 60, 35) (210, 55, 35) (180, 65, 30) (190, 60, 35)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Elaborazione canzone 62: 000255.mp3 ===

Risposta generata dal modello:

RGB: (180, 40, 20) (120, 80, 60) (200, 100, 30) (220, 50, 10) (190, 70, 25) (210, 60, 15)

--- Salvataggio dei colori RGB in CSV ---
Colori RGB salvati in 'valutazione_rgb_segmenti.csv'

Processing complete. No visualizations or JSON output were generated as per your request.
The RGB values are saved in the CSV file.
